# Fine‑tuning **mT5‑base** with **LoRA** for Informal → Formal Style Transfer (Persian)

Name:

Student ID: 

Welcome! In this assignment, you’ll build an application that converts informal Persian sentences to formal ones.

You will:

1. **Pre‑process** the *ParsMap* informal–formal corpus with the `hazm` library.  
2. **Compute** input/output *token‑length statistics* to choose sensible `max_length` values.  
3. **Fine‑tune** the multilingual T5‑base model (`google/mt5-base`) using **Low‑Rank Adaptation (LoRA)**.  
4. **Evaluate** your model with BLEU and **perplexity**.  
5. **Explore** *stochastic decoding* strategies (temperature, top‑k, nucleus) and discuss diversity vs. quality.

Fill in each **`TODO`** region with code or text.  
When you finish, submit the completed notebook with a brief discussion section at the end summarising your findings.

### Key References  

| Topic | Paper |
|-------|------------------------------|
| Corpus | *Ehsani et al.* “Developing an Informal‑Formal Persian Corpus.” 🇮🇷 |
| Model | *Xue et al.* “mT5: A Massively Multilingual Pre‑trained Text‑to‑Text Transformer.” TACL 2021 |
| Fine‑tuning | *Hu et al.* “LoRA: Low‑Rank Adaptation of Large Language Models.” ICML 2022 |
| Decoding | *Holtzman et al.* “The Curious Case of Neural Text Degeneration.” ICLR 2020 |


## 1 · Environment & Dependencies  
Run the next cell **once** (commented by default) to install the dependencies.

In [ ]:
# 🛠️ TODO (⚠️ Uncomment the next line if you are in a fresh environment)
# !pip install transformers==4.40.1 peft==0.10.0 datasets==2.19.0 evaluate==0.4.1 accelerate==0.28.0 hazm==0.9.0 sacrebleu jupyterlab tqdm

In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from hazm import Normalizer
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)
# TODO: add any other imports you need


## 2 · Data Loading & Normalisation  
Point `FILE_PATH` to the Excel file of **ParsMap** dataset.
1. Keep only the *informal* and *formal* columns.  
2. Clean each sentence with `hazm.Normalizer`.  
3. Create `train`, `validation`, and `test` splits (90 / 5 / 5 %).  


In [ ]:
# TODO ↓
FILE_PATH = "PATH/TO/ParsMap.xlsx"  # ← update this!

# 1. Load the file
df = pd.read_excel(FILE_PATH)[['inFormalForm', 'formalForm']].rename(
    columns={'inFormalForm':'input', 'formalForm':'target'}
)

# 2. Normalise
normalizer = Normalizer()
df['input']  = df['input'].apply(normalizer.normalize)
df['target'] = df['target'].apply(normalizer.normalize)

# 3. Split to HF DatasetDict
full_ds = Dataset.from_pandas(df)
full_ds = full_ds.shuffle(seed=42)
split_ds = full_ds.train_test_split(test_size=0.10, seed=42)
val_test = split_ds['test'].train_test_split(test_size=0.50, seed=42)
dataset = DatasetDict({'train': split_ds['train'],
                       'validation': val_test['train'],
                       'test': val_test['test']})
dataset

## 3 · Token‑length Statistics  
Before padding/truncation, inspect sequence lengths to decide `max_length` for **inputs** and **targets**.  
Write a helper `length_stats()` that returns *min, max, mean, 95‑percentile*.  


In [ ]:
# TODO ↓
tokenizer = AutoTokenizer.from_pretrained('google/mt5-base', use_fast=False)

def length_stats(texts):
    """Return descriptive statistics over tokenised length."""
    # YOUR CODE HERE
    raise NotImplementedError

input_stats  = length_stats(dataset['train']['input'])
target_stats = length_stats(dataset['train']['target'])

print('Input stats :', input_stats)
print('Target stats:', target_stats)

# Decide sensible values
MAX_SOURCE_LEN = ...  # TODO
MAX_TARGET_LEN = ...  # TODO


### Tokenisation function  
Complete `preprocess_function` so that it returns `input_ids`, `attention_mask`, and `labels` truncated/padded to the lengths chosen above.

In [ ]:
# TODO ↓
def preprocess_function(batch):
    # YOUR CODE HERE
    return model_inputs

tokenised_ds = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
tokenised_ds

## 4 · Model & LoRA Configuration  
Instantiate *mT5‑base* and wrap it with **LoRA**.  
Read the LoRA paper and, based on its insights and your available GPU resources, experiment with the *rank r*, `lora_alpha`, and target modules.”


In [ ]:
# TODO ↓
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    # r=,                     # TODO: tune
    # lora_alpha=,            # TODO: tune
    # target_modules=,        # TODO: tune
    lora_dropout=0.10,
    bias='none',
    task_type='SEQ_2_SEQ_LM'
)

base_model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-base')
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

## 5 · Fine‑tuning  
Define `Seq2SeqTrainingArguments` and train for **3 epochs**  
Log training loss and evaluate on the validation set each epoch.  


In [ ]:
# TODO ↓
training_args = Seq2SeqTrainingArguments(
# TODO
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest')

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_ds['train'],
    eval_dataset=tokenised_ds['validation'],
    data_collator=data_collator
)

# 🚀 Train
# trainer.train()  # ← uncomment when ready

## 6 · Inference  
Generate the *formal* version of **5 custom informal sentences** using **greedy decoding** *and* your `MAX_TARGET_LEN`.  


In [ ]:
# TODO ↓
example_inputs = [
    "واسه چی اینقدر دیر اومدی؟",
    # add 4 more
]

# Greedy decoding
# YOUR CODE HERE


## 7 · Evaluation  
Compute **BLEU** on the *test* split and report **perplexity** on *validation*.  
Explain briefly what each metric captures for this task.  


In [ ]:
# TODO ↓
import evaluate, math

bleu = evaluate.load('sacrebleu')
# 1. Generate predictions
# 2. Compute BLEU and perplexity


## 8 · Stochastic Decoding & Diversity Analysis  

Read *Holtzman et al. 2020* — *The Curious Case of Neural Text Degeneration* — to understand how different **stochastic decoding** strategies (like temperature, top‑k, and top‑p sampling) can lead to generating multiple diverse outputs from the same input prompt.

Implement these decoding strategies and experiment with several input examples to observe how the outputs vary.

In [ ]:
# TODO ↓
def sample_outputs(prompt: str,
                   num_return_sequences: int = 5,
                   temperature: float = 0.7,
                   top_k: int = 50,
                   top_p: float = 1.0):
    """Generate *num_return_sequences* diverse outputs from the fine‑tuned model."""
    # YOUR CODE HERE

prompt = "تو مطمئنی که بابا بلده گره دوتائی به کفشم بزنه وقتی که من صبحها میخوام برم مدرسه؟"
samples = sample_outputs(prompt, num_return_sequences=5, temperature=0.9, top_p=0.95)
print(*samples, sep='\n---\n')


## 9 · Discussion 

1. How did LoRA hyper‑parameters influence training stability or performance?  
2. **Deterministic vs. Stochastic Decoding**  
   Briefly explain what deterministic decoding (e.g. greedy search, beam search) and stochastic decoding (e.g. temperature sampling, top‑k/top‑p nucleus sampling) mean, drawing on Holtzman et al. 2020, *The Curious Case of Neural Text Degeneration*.
3. Suggest one improvement to the data or model that could further boost formalisation quality.  


---

### Submission Checklist ✅

- [ ] All `TODO` blocks completed.  
- [ ] Notebook runs end‑to‑end without errors (`Runtime ⇾ Restart & Run All`).  
- [ ] Answers written in the *Discussion* section.  

Good luck, and have fun experimenting! ✨
